# Module 1.1: Building Your First Agent with LangGraph

[![LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/)

**Objective:** Build a complete StateGraph application from scratch

In this exercise, you'll practice:
- Defining state schemas with TypedDict
- Implementing node functions that update state
- Using normal and conditional edges
- Compiling and testing a working graph

<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agenter-2025/blob/main/exercises/langgraph/1.1-langgraph-first-agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

## Setup

### Install dependencies

In [ ]:
%pip install openai~=2.0 --upgrade --quiet
%pip install python-dotenv~=1.0 --upgrade --quiet
%pip install langchain~=0.3 langchain_openai~=0.3 --upgrade --quiet
%pip install langgraph~=0.6 --upgrade --quiet

### Load environment variables

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    # Get API key from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    # Load from .env file for local development
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar")
        print("   → Add a secret named 'OPENAI_API_KEY'")
        print("   → Toggle 'Notebook access' to enable it")
    else:
        print("   → Create a .env file with: OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key configured!")

### Setup LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

---

## Part 1: Understanding State and Nodes

### State Schema with TypedDict

LangGraph uses TypedDict to define the state schema. This provides type hints and structure while remaining a regular dict at runtime.

In [ ]:
from typing import TypedDict, NotRequired

class NameGeneratorState(TypedDict):
    """State for our name generator agent.
    
    Fields:
        original_name: The user's input name
        cool_name: Generated cool version of the name (optional)
        greeting: Final greeting message (optional)
    """
    original_name: str
    cool_name: NotRequired[str]  # Optional field
    greeting: NotRequired[str]    # Optional field

### Node Functions

Nodes are Python functions that:
1. Receive the current state as input
2. Perform some computation
3. Return a dict with state updates

**Important:** By default, returned values OVERWRITE existing state values.

In [ ]:
def generate_cool_name(state: NameGeneratorState) -> NameGeneratorState:
    """Generate a cool version of the user's name using LLM."""
    print("---Generating Cool Name---")
    
    original = state["original_name"]
    
    prompt = f"""Generate a cool, creative variation of the name '{original}'. 
    Make it fun and memorable, but still recognizable. 
    Respond with ONLY the cool name, nothing else."""
    
    response = llm.invoke(prompt)
    cool_name = response.content.strip()
    
    print(f"Generated: {original} -> {cool_name}")
    
    # Return dict with state updates
    return {"cool_name": cool_name}


def create_happy_greeting(state: NameGeneratorState) -> NameGeneratorState:
    """Create an enthusiastic greeting."""
    print("---Creating Happy Greeting---")
    
    name = state["cool_name"]
    greeting = f"🎉 Hey there, {name}! You're awesome! Have an amazing day! 🌟"
    
    return {"greeting": greeting}


def create_sad_greeting(state: NameGeneratorState) -> NameGeneratorState:
    """Create a melancholic greeting."""
    print("---Creating Sad Greeting---")
    
    name = state["cool_name"]
    greeting = f"😔 Oh... {name}... Life is so fleeting... Anyway, hi... 💔"
    
    return {"greeting": greeting}

### Test Individual Nodes

Before building the graph, let's verify nodes work correctly:

In [ ]:
# Test the name generator node
test_state = {"original_name": "Alice"}
result = generate_cool_name(test_state)
print(f"Result: {result}")

# Test greeting node
test_state_2 = {"original_name": "Alice", "cool_name": "Cosmic Alice"}
result_2 = create_happy_greeting(test_state_2)
print(f"\nGreeting: {result_2}")

---

## Part 2: Build the Graph

### Conditional Edge Logic

Conditional edges determine routing based on state. The function returns the name of the next node.

In [ ]:
from typing import Literal

def decide_mood(state: NameGeneratorState) -> Literal["happy", "sad"]:
    """Route based on name length - even = happy, odd = sad."""
    print("---Deciding Mood---")
    
    name_length = len(state["cool_name"])
    
    if name_length % 2 == 0:
        print(f"Name length {name_length} is even -> Happy!")
        return "happy"
    else:
        print(f"Name length {name_length} is odd -> Sad...")
        return "sad"

### Construct the Graph

Now we'll build the complete workflow:

1. **START** → Generate cool name
2. Check name length (conditional edge)
3. Route to either happy or sad greeting
4. **END**

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# Initialize graph builder
builder = StateGraph(NameGeneratorState)

# Add nodes
builder.add_node("generate", generate_cool_name)
builder.add_node("happy", create_happy_greeting)
builder.add_node("sad", create_sad_greeting)

# Add edges
builder.add_edge(START, "generate")

# Conditional edge: route based on name length
builder.add_conditional_edges(
    "generate",
    decide_mood,
    {
        "happy": "happy",
        "sad": "sad"
    }
)

builder.add_edge("happy", END)
builder.add_edge("sad", END)

# Compile the graph
graph = builder.compile()

# Visualize
display(Image(graph.get_graph().draw_mermaid_png()))

---

## Part 3: Test and Iterate

### Invoke the Graph

In [ ]:
# Test with different names
test_names = ["Alice", "Bob", "Charlie", "Diana"]

for name in test_names:
    print(f"\n{'='*60}")
    print(f"Testing with: {name}")
    print(f"{'='*60}")
    
    result = graph.invoke({"original_name": name})
    
    print(f"\n✨ Final State:")
    print(f"   Original: {result['original_name']}")
    print(f"   Cool Name: {result['cool_name']}")
    print(f"   Greeting: {result['greeting']}")

### Stream the Graph (Optional)

See each step of execution:

In [ ]:
print("\n🔄 Streaming execution:\n")

for chunk in graph.stream({"original_name": "Emma"}):
    print(f"Step: {chunk}")
    print()

---

## 🎯 Exercise: Build Your Own Graph!

**Challenge:** Create a "Job Title Generator" that:

1. Takes a person's hobby as input
2. Generates a creative job title based on the hobby
3. Routes based on title coolness:
   - If title contains "Chief" or "Wizard" → enthusiastic response
   - Otherwise → standard response
4. Returns final message

### Template

In [ ]:
# STATE
class JobTitleState(TypedDict):
    hobby: str
    job_title: NotRequired[str]
    response: NotRequired[str]

# NODES
def generate_job_title(state: JobTitleState) -> JobTitleState:
    print("---Generating Job Title---")
    # TODO: Use LLM to generate creative job title from hobby
    # Example prompt: "Create a creative, fun job title for someone whose hobby is {hobby}"
    return {"job_title": "TODO"}

def enthusiastic_response(state: JobTitleState) -> JobTitleState:
    print("---Enthusiastic Response---")
    # TODO: Create enthusiastic message
    return {"response": "TODO"}

def standard_response(state: JobTitleState) -> JobTitleState:
    print("---Standard Response---")
    # TODO: Create standard message
    return {"response": "TODO"}

# CONDITIONAL EDGE
def check_coolness(state: JobTitleState) -> Literal["enthusiastic", "standard"]:
    # TODO: Check if title contains "Chief" or "Wizard"
    title = state["job_title"].lower()
    if "chief" in title or "wizard" in title:
        return "enthusiastic"
    return "standard"

# GRAPH
# TODO: Build the graph
# builder_exercise = StateGraph(JobTitleState)
# ...
# graph_exercise = builder_exercise.compile()

# Test it!
# result = graph_exercise.invoke({"hobby": "gardening"})
# print(result)

---

## Key Takeaways

✅ **State Schema**: Use TypedDict to define structured state  
✅ **Nodes**: Python functions that receive state and return updates  
✅ **Edges**: Connect nodes (normal or conditional)  
✅ **Conditional Logic**: Route based on state values  
✅ **Compilation**: Always compile before using the graph  

### Next Steps

- Module 1.7: LLM-based routing for dynamic decision making
- Module 1.8: Tool calling with ReAct pattern

---

## Additional Resources

- [LangGraph Low-Level Concepts](https://langchain-ai.github.io/langgraph/concepts/low_level/)
- [StateGraph Documentation](https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)